# Practicum Lightning

https://lightning.ai/docs/pytorch/stable/starter/introduction.html#define-a-lightningmodule <br>
Lightning in 15 minutes<br>
PyTorch Lightning is the deep learning framework with “batteries included” for professional AI researchers and machine learning engineers who need maximal flexibility while super-charging performance at scale.<br>
<br>
Lightning organizes PyTorch code to remove boilerplate and unlock scalability.<br>

## 1: Install PyTorch Lightning

For pip users: pip install lightning<br>

## 2: Define a LightningModule
<br>
A LightningModule enables your PyTorch nn.Module to play together in complex ways inside the training_step (there is also an optional validation_step and test_step).

In [1]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning.pytorch as pl

# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))


# define the LightningModule
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

## 3: Define a dataset

Lightning supports ANY iterable (DataLoader, numpy, etc…) for the train/val/test/predict splits.

In [2]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

100%|███████████████████████████| 9912422/9912422 [00:00<00:00, 10192076.29it/s]


Extracting /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/MNIST/raw/train-images-idx3-ubyte.gz to /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/MNIST/raw



100%|████████████████████████████████| 28881/28881 [00:00<00:00, 7945929.41it/s]


Extracting /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/MNIST/raw/train-labels-idx1-ubyte.gz to /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/MNIST/raw



100%|████████████████████████████| 1648877/1648877 [00:00<00:00, 5772054.83it/s]


Extracting /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/MNIST/raw



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 3672037.16it/s]

Extracting /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/MNIST/raw



## 4: Train the model

The Lightning Trainer “mixes” any LightningModule with any dataset and abstracts away all the engineering complexity needed for scale.

In [7]:
# You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
import torch
torch.set_float32_matmul_precision('medium' | 'high')

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [3]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/pans/workspace/MakeAIWork2/practica/07_ml_frameworks/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
/home/pans/minicon

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


GPU (Graphical Processing Unit) available: True (cuda), used: True
TPU (Tensor PU) available: False, using: 0 TPU cores
IPU (Intelligence PU) available: False, using: 0 IPUs
HPU (Havana Gaudi PU )available: False, using: 0 HPUs
/home/pien/miniconda3/envs/miw/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/pien/Workspace/MakeAIWork2/practica/Practicum Lightning/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
/home/pien/miniconda3/envs/miw/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


### The Lightning Trainer automates 40+ tricks including:

- Epoch and batch iteration
- optimizer.step(), loss.backward(), optimizer.zero_grad() calls
- Calling of model.eval(), enabling/disabling grads during evaluation
- Checkpoint Saving and Loading
- Tensorboard (see loggers options)
- Multi-GPU support
- TPU
- 16-bit precision AMP support

## 5: Use the model

Once you’ve trained the model you can export to onnx, torchscript and put it into production or simply load the weights and run predictions.

In [4]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = Tensor(4, 28 * 28)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[-1.2548e+36,  3.1359e+36,  4.9143e+35],
        [        nan,         nan,         nan],
        [-2.7249e+36, -3.8274e+35,  4.7455e+35],
        [-2.4258e+29, -2.9490e+30, -2.0074e+30]], grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡


## 6: Visualize training

If you have tensorboard installed, you can use it for visualizing experiments. TENSORBOARD is niet geinstalleerd!!

Run this on your commandline and open your browser to http://localhost:6006/